<a href="https://colab.research.google.com/github/mrospond/kik/blob/main/W03_KiK_kodowanie_%C5%BAr%C3%B3d%C5%82owe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Potrzebne biblioteki...

# Do sortowania prawdopodobieństw będziemy potrzebować kolejki priorytetowej
import heapq

# Rysowanie standardowych wykresów
import matplotlib.pyplot as plt
# Obróbka statystyczna przy rysowaniu wykresów i formatowaniu ich wyglądu
import seaborn as sns
sns.set()

# Obliczenia matematyczne, dla nas głównie logarytm
import math

# Praca z wektorami i macierzami
import numpy as np

# Eleganckie formatowanie wyświetlanych danych
import pprint

# Obliczenia probabilistyczne
# Entropia Shannona i dywergencja Kullbacka-Leiblera
from scipy.stats import entropy

# Obsługa plików
## Pobieranie
import requests
## Rozpakowywanie
import zipfile

# Zliczanie znaków w ciągu
from collections import Counter

## Średnia (oczekiwana) długość słowa kodowego

Mamy dany zbiór wiadomości $M$ i reprezentujemy je z użyciem kodu $\mathcal{C}$, np. dla kodu ASCII:

$$ m_1 = \text{A} \quad \mathcal{C}(m_1) = \mathcal{C}(\text{A}) = 01000001$$

W kodowaniu źródłowym przy ustalonym kodzie:
*   kodowane wiadomości: $\{m_1,m_2,\dots,m_N\} = M$
*   prawdopodobieństwo wiadomości: $p_i = \Pr\{m_i\}$
*   długość słowa kodowego reprezentującego wiadomość: $\ell_i = \mathcal{C}(m_i)$
możemy policzyć średnią długość słowa kodowego (ciekawe, gdy mamy różne prawdopodobieństwa wiadomości i różne długości słów kodowych):

$$ \overline{L} = \sum\limits_{i=1}^N{p_i\ell_i} $$

Długość słowa kodowego mówi nam, na ile kod jest efektywny z punktu widzenia transmisyjnego czy np. przechowywania w pamięci.

In [ ]:
#@markdown Funkcja do liczenia średniej długości słowa

def srednia_dlugosc(czestotliwosci, slowa, wyswietlanie=False):
  suma = 0
  ile_zer = 0
  ile_jedynek = 0
  for litera in czestotliwosci:
    prawdopodobienstwo = czestotliwosci[litera]
    slowo = slowa[litera]
    dlugosc = len(slowo)
    if wyswietlanie:
      print(f"Dla litery {litera} o prawdopodobieństwie {prawdopodobienstwo} kodujemy słowem {slowo} o długości {dlugosc}")
    suma += prawdopodobienstwo * dlugosc
    ile_zer += prawdopodobienstwo * Counter(slowo)['0']
    ile_jedynek += prawdopodobienstwo * Counter(slowo)['1']

  print("-" * 100)
  print(f"Średnia długość słowa kodowego: {suma}")
  print(f"Średnio mamy zer: {ile_zer/suma} i jedynek: {ile_jedynek/suma}")

  return suma

In [ ]:
#@title { run: "auto" }
#@markdown * Źródło z sześcioma wiadomościami $A, B, C, D, E, F$
#@markdown * Prawdopodobieństwa: $\frac{1}{2},\frac{1}{4},\frac{1}{16},\frac{1}{16},\frac{1}{16},\frac{1}{16}$

kod_A = "0"  #@param {type: "string"}
kod_B = "10"  #@param {type: "string"}
kod_C = "1101"  #@param {type: "string"}
kod_D = "1110"  #@param {type: "string"}
kod_E = "1111"  #@param {type: "string"}
kod_F = "1100"  #@param {type: "string"}

wiadomosci = ["A", "B", "C", "D", "E", "F"]
prawd = [0.5, 0.25, 0.0625, 0.0625, 0.0625, 0.0625]
rozklad_prawdopodobienstwa = np.array(prawd)

H = entropy(rozklad_prawdopodobienstwa, base = 2)

print(f'Entropia H({prawd}) = {H} bit/wiad.')

czest = dict(zip(wiadomosci,prawd))
slow = dict(zip(wiadomosci,[kod_A,kod_B,kod_C,kod_D,kod_E,kod_F]))
suma = srednia_dlugosc(czest,slow)

Entropia H([0.5, 0.25, 0.0625, 0.0625, 0.0625, 0.0625]) = 2.0 bit/wiad.
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 2.0
Średnio mamy zer: 0.5 i jedynek: 0.5


In [ ]:
#@markdown Dla języka angielskiego i pewnego kodu jednoznacznie dekodowalnego

Eng_freq = {
  'A': 0.0812,
  'B': 0.0149,
  'C': 0.0271,
  'D': 0.0432,
  'E': 0.12,
  'F': 0.023,
  'G': 0.0203,
  'H': 0.0592,
  'I': 0.0731,
  'J': 0.001,
  'K': 0.0069,
  'L': 0.0398,
  'M': 0.0261,
  'N': 0.0695,
  'O': 0.0768,
  'P': 0.0182,
  'Q': 0.0011,
  'R': 0.0602,
  'S': 0.0628,
  'T': 0.091,
  'U': 0.0288,
  'V': 0.0111,
  'W': 0.0209,
  'X': 0.0017,
  'Y': 0.0211,
  'Z': 0.0007
}

# Tworzymy kod
Kod = {}
licznik = 0
for litera in Eng_freq:
  Kod[litera] = '1'*licznik + '0'
  licznik += 1

print("Nasz kod:")
print("-" * 100)
pprint.pprint(Kod)

Nasz kod:
----------------------------------------------------------------------------------------------------
{'A': '0',
 'B': '10',
 'C': '110',
 'D': '1110',
 'E': '11110',
 'F': '111110',
 'G': '1111110',
 'H': '11111110',
 'I': '111111110',
 'J': '1111111110',
 'K': '11111111110',
 'L': '111111111110',
 'M': '1111111111110',
 'N': '11111111111110',
 'O': '111111111111110',
 'P': '1111111111111110',
 'Q': '11111111111111110',
 'R': '111111111111111110',
 'S': '1111111111111111110',
 'T': '11111111111111111110',
 'U': '111111111111111111110',
 'V': '1111111111111111111110',
 'W': '11111111111111111111110',
 'X': '111111111111111111111110',
 'Y': '1111111111111111111111110',
 'Z': '11111111111111111111111110'}


Jak taki kod się zachowuje z punktu widzenia średniej długości?

In [ ]:
#@markdown Średnia długość słowa kodowego

suma = srednia_dlugosc(Eng_freq, Kod)

----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 11.727400000000001
Średnio mamy zer: 0.08524481129662159 i jedynek: 0.9147551887033782


Da się lepiej? Na pewno można trochę sprytniej...

In [ ]:
#@markdown Podmieńmy...

Kod['E'], Kod['B'] = Kod['B'], Kod['E']

pprint.pprint(Kod)
suma = srednia_dlugosc(Eng_freq, Kod)

{'A': '0',
 'B': '11110',
 'C': '110',
 'D': '1110',
 'E': '10',
 'F': '111110',
 'G': '1111110',
 'H': '11111110',
 'I': '111111110',
 'J': '1111111110',
 'K': '11111111110',
 'L': '111111111110',
 'M': '1111111111110',
 'N': '11111111111110',
 'O': '111111111111110',
 'P': '1111111111111110',
 'Q': '11111111111111110',
 'R': '111111111111111110',
 'S': '1111111111111111110',
 'T': '11111111111111111110',
 'U': '111111111111111111110',
 'V': '1111111111111111111110',
 'W': '11111111111111111111110',
 'X': '111111111111111111111110',
 'Y': '1111111111111111111111110',
 'Z': '11111111111111111111111110'}
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 11.4121
Średnio mamy zer: 0.08760000350505166 i jedynek: 0.9123999964949482


W przypadku tego rodzaju kodu najsprytniej byłoby posortować po prawdopodobieństwach i wtedy przydzielić długości słów kodowych:

In [ ]:
#@markdown Najlepszy tego rodzaju kod

posortowane = sorted(Eng_freq.items(), key=lambda x: x[1], reverse = True)
# print(posortowane)

klucze = list(zip(*posortowane))[0]
# print(klucze)

Kod1 = {}
licznik = 0
for litera in klucze:
  Kod1[litera] = '1'*licznik + '0'
  licznik += 1

print("-" * 100)
print("Nasz nowy, lepszy kod:")
print("-" * 100)
pprint.pprint(Kod1)
suma = srednia_dlugosc(Eng_freq, Kod1)

----------------------------------------------------------------------------------------------------
Nasz nowy, lepszy kod:
----------------------------------------------------------------------------------------------------
{'A': '110',
 'B': '11111111111111111110',
 'C': '1111111111110',
 'D': '1111111110',
 'E': '0',
 'F': '111111111111110',
 'G': '111111111111111110',
 'H': '111111110',
 'I': '11110',
 'J': '1111111111111111111111110',
 'K': '1111111111111111111110',
 'L': '11111111110',
 'M': '11111111111110',
 'N': '111110',
 'O': '1110',
 'P': '1111111111111111110',
 'Q': '111111111111111111111110',
 'R': '11111110',
 'S': '1111110',
 'T': '10',
 'U': '111111111110',
 'V': '111111111111111111110',
 'W': '11111111111111110',
 'X': '11111111111111111111110',
 'Y': '1111111111111110',
 'Z': '11111111111111111111111110'}
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 7.5631
Średnio mamy zer: 0.1321

I co z tego? Sprawdźmy...

In [ ]:
#@markdown Policzymy najpierw entropię dla języka angielskiego (na poziomie liter)

print(f"Entropia dla języka angielskiego wynosi {entropy(list(Eng_freq.values()), base = 2)}")

Entropia dla języka angielskiego wynosi 4.181610514870444


Ciekawe czy to widać na dowolnym tekście...

In [ ]:
#@title Użyjemy korpusu języka angielskiego (przykładowy tekst)

url = 'http://corpus.canterbury.ac.nz/resources/cantrbry.zip'
r = requests.get(url, allow_redirects=True)
open('cantrbry.zip', 'wb').write(r.content)
with zipfile.ZipFile('cantrbry.zip', 'r') as zip_ref:
    zip_ref.extractall('cantrbry')

with open('cantrbry/alice29.txt', encoding='utf8') as f:
    text = f.read()
f.close()

text = text.upper()
print(text[:100])





                ALICE'S ADVENTURES IN WONDERLAND

                          LEWIS CARROLL

     


Spróbujmy zakodować Alicję naszym kodem:

In [ ]:
#@markdown * Kodowanie kodami Kod i Kod1
#@markdown * Możemy mieć odchyłki średniej długości (eksperymentalnej) oraz oczekiwanej długości (teoretycznej):
#@markdown     * średnia z eksperymentu zawsze może różnić się od wartości teoretycznej (oczekiwanej z rozkładu);
#@markdown     * prawdopodobieństwa użyte do wyliczenia wartości oczekiwanej mogą nieco odbiegać od statystyk Alicji.

def kodowanie(tekst, kod):

  symbole = ''
  ile_liter = 0
  for char in text:
    if char in Eng_freq.keys():
      ile_liter += 1
      symbole = symbole + kod[char]

  sr_dlugosc = len(symbole)/ile_liter

  return symbole, sr_dlugosc

symb, sr_dl = kodowanie(text[:100], Kod)
print(text[:100], "\n", symb)
print(f"Średnia długość dla tej sekwencji 100 wiadomości: {sr_dl}")

print("Pierwszy kod:")
_, sr_dl = kodowanie(text, Kod)
print(f"Średnia długość dla Alicji: {sr_dl}, a wyliczona dla zdefiniowanego kodu: {srednia_dlugosc(Eng_freq, Kod)}")

print("Kod sprytniejszy:")
_, sr_dl = kodowanie(text, Kod1)
print(f"Średnia długość dla Alicji: {sr_dl}, a wyliczona dla zdefiniowanego kodu: {srednia_dlugosc(Eng_freq, Kod1)}")





                ALICE'S ADVENTURES IN WONDERLAND

                          LEWIS CARROLL

      
 ...
Średnia długość dla tej sekwencji 100 wiadomości: 11.357305395339333
Pierwszy kod:
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 11.4121
Średnio mamy zer: 0.08760000350505166 i jedynek: 0.9123999964949482
Średnia długość dla Alicji: 11.357305395339333, a wyliczona dla zdefiniowanego kodu: 11.4121
Kod sprytniejszy:
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 7.5631
Średnio mamy zer: 0.13218124842987666 i jedynek: 0.8678187515701236
Średnia długość dla Alicji: 7.477713691288882, a wyliczona dla zdefiniowanego kodu: 7.5631


Czy jest jeszcze sprytniejszy kod?

# Kod zwięzły: binarny Kod Huffmana

Kod Huffmana daje najkrótszy możliwy kod dla zadanego zestawu prawdopodobieństw (częstotliwości) wiadomości.

In [ ]:
#@markdown Kodowanie Huffmana, ze strony [Geeks for Geeks](https://www.geeksforgeeks.org/huffman-coding-greedy-algo-3/)

def huffman_code(letters_prob):

  kod_huffmana = {}

  class node:
      def __init__(self, freq, symbol, left=None, right=None):
          # frequency of symbol
          self.freq = freq
  
          # symbol name (character)
          self.symbol = symbol
  
          # node left of current node
          self.left = left
  
          # node right of current node
          self.right = right
  
          # tree direction (0/1)
          self.huff = ''
          
      def __lt__(self, nxt):
          return self.freq < nxt.freq
          
  
  # utility function to print Huffman
  # codes for all symbols in the newly
  # created Huffman tree
  def printNodes(node, val=''):
      
      # huffman code for current node
      newVal = val + str(node.huff)
  
      # if node is not an edge node
      # then traverse inside it
      if(node.left):
          printNodes(node.left, newVal)
      if(node.right):
          printNodes(node.right, newVal)
  
          # if node is edge node then
          # display its huffman code
      if(not node.left and not node.right):
          print(f"{node.symbol} -> {newVal}")
          kod_huffmana[node.symbol] = newVal
  
  # we skip characters with zero frequency/probability
  non_zero_freq = {k: v for k, v in letters_prob.items() if v != 0}  

  # characters for huffman tree
  chars = list(non_zero_freq.keys())
  
  # frequency of characters
  freq = list(non_zero_freq.values())
  
  # list containing unused nodes
  nodes = []
  
  # converting characters and frequencies
  # into Huffman tree nodes
  for x in range(len(chars)):
      heapq.heappush(nodes, node(freq[x], chars[x]))
  
  while len(nodes) > 1:
      
      # sort all the nodes in ascending order
      # based on their frequency
      left = heapq.heappop(nodes)
      right = heapq.heappop(nodes)
  
      # assign directional value to these nodes
      # binary code!
      left.huff = 0
      right.huff = 1
  
      # combine the TWO SMALLEST nodes to create
      # new node as their parent
      newNode = node(left.freq+right.freq, left.symbol+right.symbol, left, right)
  
      heapq.heappush(nodes, newNode)
  
  # Huffman Tree is ready!
  print("Reprezentacja kodu: ")
  printNodes(nodes[0])

  print()
  print("Słownik z kodem: ")
  pprint.pprint(kod_huffmana)

  return kod_huffmana

In [ ]:
#@markdown Kodujemy dla liter języka angielskiego

Kod2 = huffman_code(Eng_freq)

Reprezentacja kodu: 
T -> 000
Y -> 00100
V -> 001010
Z -> 001011000
J -> 001011001
Q -> 001011010
X -> 001011011
K -> 0010111
F -> 00110
M -> 00111
C -> 01000
U -> 01001
H -> 0101
E -> 011
R -> 1000
S -> 1001
N -> 1010
B -> 101100
P -> 101101
L -> 10111
I -> 1100
O -> 1101
A -> 1110
G -> 111100
W -> 111101
D -> 11111

Słownik z kodem: 
{'A': '1110',
 'B': '101100',
 'C': '01000',
 'D': '11111',
 'E': '011',
 'F': '00110',
 'G': '111100',
 'H': '0101',
 'I': '1100',
 'J': '001011001',
 'K': '0010111',
 'L': '10111',
 'M': '00111',
 'N': '1010',
 'O': '1101',
 'P': '101101',
 'Q': '001011010',
 'R': '1000',
 'S': '1001',
 'T': '000',
 'U': '01001',
 'V': '001010',
 'W': '111101',
 'X': '001011011',
 'Y': '00100',
 'Z': '001011000'}


In [ ]:
#@title A jaka średnia długość?

print("Kod Huffmana:")
_, sr_dl = kodowanie(text, Kod2)
print(f"Średnia długość dla Alicji: {sr_dl}, a wyliczona dla zdefiniowanego kodu: {srednia_dlugosc(Eng_freq, Kod2)}")

Kod Huffmana:
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 4.2109000000000005
Średnio mamy zer: 0.45838181861359795 i jedynek: 0.5416181813864019
Średnia długość dla Alicji: 4.20518821923152, a wyliczona dla zdefiniowanego kodu: 4.2109000000000005


Jak już mamy taką śliczną funkcję do liczenia kodu Huffmana, to się pobawmy na źródle o ośmiu wiadomościach:

In [ ]:
#@title { run: "auto" }
#@markdown Źródło z ośmioma wiadomościami

#@markdown Prawdopodobieństwa $p_1,\dots,p_8$ (automatycznie wyliczamy $p_8 = 1-\sum_{i=1}^7{p_i}$):
p1 = 0.54  #@param {type: "slider", min:0, max:1, step:0.01}
p2 = 0.25  #@param {type: "slider", min:0, max:1, step:0.01}
p3 = 0.08  #@param {type: "slider", min:0, max:1, step:0.01}
p4 = 0  #@param {type: "slider", min:0, max:1, step:0.01}
p5 = 0  #@param {type: "slider", min:0, max:1, step:0.01}
p6 = 0  #@param {type: "slider", min:0, max:1, step:0.01}
p7 = 0  #@param {type: "slider", min:0, max:1, step:0.01}

p8 = 1 - p1 - p2 - p3 - p4 - p5 - p6 - p7

assert p8 >= 0, "Suma prawdopodobieństw nie może przekracać 1.0!"


prawd = [p1, p2, p3, p4, p5, p6, p7, p8]
rozklad_prawdopodobienstwa = np.array(prawd)

H = entropy(rozklad_prawdopodobienstwa, base = 2)

print(f'Entropia H({prawd}) = {H} bit/wiad.')

Entropia H([0.25, 0.25, 0.08, 0, 0, 0, 0, 0.42]) = 1.8171547773202832 bit/wiad.


In [ ]:
#@markdown To sobie znajdźmy kod Huffmana dla źródła

def robimy_kod(prawd, k=1):
  freq = dict(zip(list(range(1,len(prawd)+1)),prawd))
  let_freq = {k: v for k, v in freq.items() if v != 0}
  kf = huffman_code(let_freq)
  srednia_dlugosc(let_freq,kf)
  H = entropy(list(freq.values()),base=2)
  print(f"Entropia źrodła: {H} bit.")
  if k>1: 
    print(f"Entropia źrodła na wiadomość: {H/k} bit/wiad.")

robimy_kod(rozklad_prawdopodobienstwa)

Reprezentacja kodu: 
8 -> 0
2 -> 10
3 -> 110
1 -> 111

Słownik z kodem: 
{1: '111', 2: '10', 3: '110', 8: '0'}
----------------------------------------------------------------------------------------------------
Średnia długość słowa kodowego: 1.91
Średnio mamy zer: 0.39267015706806285 i jedynek: 0.6073298429319371
Entropia źrodła: 1.8171547773202832 bit.


Co będzie jak rozszerzymy to źródło?

In [ ]:
#@title { run: "auto" }
#@markdown * Źródło rozszerzone i jego kod zwięzły
#@markdown * Najlepiej pozytywne efekty rozszerzania widać, gdy mamy skośne rozkłady prawdopodobieńśtwa (czyli np. z jedną dominującą wiadomością)

k = 3  #@param {type: "slider", min:1, max:10, step:1}

prawdo = np.array([p for p in prawd if p>0])
prawdop = prawdo

for i in range(k-1):
  macierz = prawdop[:, np.newaxis] * prawdo
  prawdop = macierz.flatten()

print(prawdop)

if k>1:
  print(f"Rozszerzenie bezpamięciowe {k}-krotne.")
robimy_kod(prawdop,k)

[0.015625 0.015625 0.005    0.02625  0.015625 0.015625 0.005    0.02625
 0.005    0.005    0.0016   0.0084   0.02625  0.02625  0.0084   0.0441
 0.015625 0.015625 0.005    0.02625  0.015625 0.015625 0.005    0.02625
 0.005    0.005    0.0016   0.0084   0.02625  0.02625  0.0084   0.0441
 0.005    0.005    0.0016   0.0084   0.005    0.005    0.0016   0.0084
 0.0016   0.0016   0.000512 0.002688 0.0084   0.0084   0.002688 0.014112
 0.02625  0.02625  0.0084   0.0441   0.02625  0.02625  0.0084   0.0441
 0.0084   0.0084   0.002688 0.014112 0.0441   0.0441   0.014112 0.074088]
Rozszerzenie bezpamięciowe 3-krotne.
Reprezentacja kodu: 
52 -> 0000
10 -> 0001000
9 -> 0001001
59 -> 00010100
44 -> 00010101
47 -> 00010110
11 -> 000101110
35 -> 000101111
30 -> 00011
49 -> 00100
50 -> 00101
29 -> 00110
54 -> 00111
14 -> 01000
24 -> 01001
4 -> 01010
53 -> 01011
13 -> 01100
20 -> 01101
8 -> 01110
48 -> 011110
60 -> 011111
63 -> 100000
41 -> 100001000
42 -> 100001001
39 -> 100001010
43 -> 1000010110
27 -> 